# **PyTorch NN Module**

The torch.nn module in PyTorch is a core library that provides a wide array of classes and
functions designed to help developers build neural networks efficiently and effectively. It
abstracts the complexity of creating and training neural networks by offering pre-built layers,
loss functions, activation functions, and other utilities, enabling you to focus on designing and
experimenting with model architectures.

**Key Components of torch.nn:**
1. Modules (Layers):
    - `nn.Module`: The base class for all neural network modules. Your custom models and
    layers should subclass this class.
    - Common Layers: Includes layers like `nn.Linear` (fully connected layer), `nn.Conv2d`
    (convolutional layer), `nn.LSTM` (recurrent layer), and many others.

2. Activation Functions:
    - Functions like `nn.ReLU`, `nn.Sigmoid`, and `nn.Tanh` introduce non-linearities to the model, allowing it to learn complex patterns.

3. Loss Functions:
    - Provides loss functions such as `nn.CrossEntropyLoss`, `nn.MSELoss`, and `nn.NLLLoss` to quantify the difference between the model's predictions and the actual targets.

4. Container Modules:
    - `nn.Sequential`: A sequential container to stack layers in order.

5. Regularization and Dropout:
    - Layers like `nn.Dropout` and `nn.BatchNorm2d` help prevent overfitting and improve the model's ability to generalize to new data.

## **Import Dependencies**

In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import torch
from torch import nn
from torchinfo import summary
from torchmetrics import Accuracy

import warnings
warnings.filterwarnings('ignore')

## **Read the Dataset**

In [3]:
# Load the breast cancer dataset using Pandas
data = pd.read_csv(r"D:\GITHUB\pytorch-for-deep-Learning-and-machine-learning\datasets\breast_cancer_data.csv")
print(data.shape)
data.head()

(569, 33)


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


## **Data Pre-processing**

### **Data Cleaning**

In [4]:
# Drop the irrelevant columns
data.drop(columns=['id', 'Unnamed: 32'], inplace=True)
print(data.shape)
data.head()

(569, 31)


,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


### **Train-Test Split**

In [5]:
# Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(columns=['diagnosis']),
    data['diagnosis'],
    test_size=0.3,
    random_state=42
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((398, 30), (171, 30), (398,), (171,))

### **Feature Scaling**

In [6]:
# Print the column information
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 398 entries, 149 to 102
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   radius_mean              398 non-null    float64
 1   texture_mean             398 non-null    float64
 2   perimeter_mean           398 non-null    float64
 3   area_mean                398 non-null    float64
 4   smoothness_mean          398 non-null    float64
 5   compactness_mean         398 non-null    float64
 6   concavity_mean           398 non-null    float64
 7   concave points_mean      398 non-null    float64
 8   symmetry_mean            398 non-null    float64
 9   fractal_dimension_mean   398 non-null    float64
 10  radius_se                398 non-null    float64
 11  texture_se               398 non-null    float64
 12  perimeter_se             398 non-null    float64
 13  area_se                  398 non-null    float64
 14  smoothness_se            398 

In [8]:
# Scale the input variables using standarad scaler
scaler = StandardScaler()
X_train_scaled =scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(X_train_scaled.shape, X_test_scaled.shape)

(398, 30) (171, 30)


### **Label Encoding**

In [9]:
# Encode the target variable using label encoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

print(y_train_encoded.shape, y_test_encoded.shape)

(398,) (171,)


### **Convert NumPy Arrays to PyTorch Tensor**

In [18]:
X_train_tensor = torch.from_numpy(X_train_scaled).type(torch.float32)
X_test_tensor = torch.from_numpy(X_test_scaled).type(torch.float32)
y_train_tensor = torch.from_numpy(y_train_encoded).type(torch.float32)
y_test_tensor = torch.from_numpy(y_test_encoded).type(torch.float32)

print(X_train_tensor.shape, X_test_tensor.shape, y_train_encoded.shape, y_test_encoded.shape)

torch.Size([398, 30]) torch.Size([171, 30]) (398,) (171,)


## **Build a Simple NN Model**

In [33]:
# Create a simple neural network model with a single node
class MySimpleNN(nn.Module):

    def __init__(self, num_features):
        super().__init__()

        self.linear = nn.Linear(num_features, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, X: torch.Tensor):
        out = self.linear(X)
        out = self.sigmoid(out)

        return out

### **Training Pipeline**

In [38]:
# Create an object of the model
model = MySimpleNN(X_train_tensor.shape[1])
# Show the model summary
summary(model)

Layer (type:depth-idx)                   Param #
MySimpleNN                               --
├─Linear: 1-1                            31
├─Sigmoid: 1-2                           --
Total params: 31
Trainable params: 31
Non-trainable params: 0

In [39]:
# Set the learning rate and number of epoch
lr = 0.1 # learning rate
epochs = 25

# Define a loss function and an optimizer
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [50]:
# Define a loop
for epoch in range(epochs):

    # Forward pass
    y_pred = model(X_train_tensor)

    # Loss calculation
    loss = loss_fn(y_pred.squeeze(), y_train_tensor)

    # Zero gradients
    optimizer.zero_grad()

    # Backward pass
    loss.backward()

    # Parameters update
    optimizer.step()

    # Print loss in epoch
    print(f'Epoch: {epoch + 1}, Loss: {loss}')

Epoch: 1, Loss: 0.6484077572822571
Epoch: 2, Loss: 0.31392422318458557
Epoch: 3, Loss: 0.20721401274204254
Epoch: 4, Loss: 0.1574023962020874
Epoch: 5, Loss: 0.12950704991817474
Epoch: 6, Loss: 0.11386612802743912
Epoch: 7, Loss: 0.10390742868185043
Epoch: 8, Loss: 0.09678122401237488
Epoch: 9, Loss: 0.09131297469139099
Epoch: 10, Loss: 0.08692934364080429
Epoch: 11, Loss: 0.08334304392337799
Epoch: 12, Loss: 0.08041228353977203
Epoch: 13, Loss: 0.078069306910038
Epoch: 14, Loss: 0.07627280801534653
Epoch: 15, Loss: 0.07496736943721771
Epoch: 16, Loss: 0.07406269013881683
Epoch: 17, Loss: 0.07344499975442886
Epoch: 18, Loss: 0.07300373166799545
Epoch: 19, Loss: 0.07264965027570724
Epoch: 20, Loss: 0.07231933623552322
Epoch: 21, Loss: 0.0719723030924797
Epoch: 22, Loss: 0.07158651947975159
Epoch: 23, Loss: 0.07115405052900314
Epoch: 24, Loss: 0.07067767530679703
Epoch: 25, Loss: 0.0701679214835167


In [51]:
# Print the model weights and bias
print('Model weights:')
print(model.linear.weight)

print('Model bias:')
print(model.linear.bias)

Model weights:
Parameter containing:
tensor([[ 0.9443,  1.1426,  0.7460,  0.6416,  0.5991,  0.4249,  0.8939,  0.7191,
          0.3031, -1.1574,  1.1007, -0.1218,  0.9625,  1.0889,  0.3460, -0.8296,
         -0.2787,  0.5059, -0.5444, -1.1104,  0.9189,  1.2887,  0.8699,  0.9161,
          0.9445,  0.5765,  0.6741,  0.6480,  0.8668,  0.1680]],
       requires_grad=True)
Model bias:
Parameter containing:
tensor([-0.9377], requires_grad=True)


### **Model Evaluation**

In [54]:
# Make predictions on testing data
with torch.no_grad():
    y_pred = model(X_test_tensor)

# Calculate the accuracy using torchmetrics
accuracy = Accuracy(task='binary')
print('Accuracy on testing data:', accuracy(y_pred.squeeze(), y_test_tensor).item())

Accuracy on testing data: 0.9941520690917969


## **Build a NN Model with a Hidden Layer**

In [55]:
# Create a neural network with a hidden layers
class MyComplexNN(nn.Module):

    def __init__(self, num_feature):
        super().__init__()
        self.linear1 = nn.Linear(in_features=num_feature, out_features=3)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(in_features=3, out_features=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, X: torch.Tensor):
        out = self.linear1(X)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.sigmoid(out)

        return out

In [57]:
# Build the same model with sequential container
class MyComplexNN(nn.Module):

    def __init__(self, num_features):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(num_features, 3), # input layer
            nn.ReLU(), # activation
            nn.Linear(3, 1), # hidden layer
            nn.Sigmoid()
        )

    def forward(self, X: torch.Tensor):
        out = self.network(X)

        return out

### **Training Pipeline**

In [87]:
# Create an object of the model class
model = MyComplexNN(X_train_tensor.shape[1])
# Print the model summary
summary(model)

Layer (type:depth-idx)                   Param #
MyComplexNN                              --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       93
│    └─ReLU: 2-2                         --
│    └─Linear: 2-3                       4
│    └─Sigmoid: 2-4                      --
Total params: 97
Trainable params: 97
Non-trainable params: 0

In [88]:
# # Set the learning rate and number of epoch
lr = 0.01
epochs = 25

# Define a loss function and an optimizer
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [89]:
# Define a loop
for epoch in range(epochs):

    # Forward pass
    y_pred = model(X_train_tensor)

    # Loss calculation
    loss = loss_fn(y_pred.squeeze(), y_train_tensor)

    # Zero gradients
    optimizer.zero_grad()

    # Backward pass
    loss.backward()

    # Parameters update
    optimizer.step()

    # Print epoch loss
    print(f'Epoch: {epoch + 1}, Loss: {loss}')

Epoch: 1, Loss: 0.7183933854103088
Epoch: 2, Loss: 0.7053613066673279
Epoch: 3, Loss: 0.6943788528442383
Epoch: 4, Loss: 0.6850125193595886
Epoch: 5, Loss: 0.6758115887641907
Epoch: 6, Loss: 0.6664143204689026
Epoch: 7, Loss: 0.6565479636192322
Epoch: 8, Loss: 0.646085798740387
Epoch: 9, Loss: 0.6351872682571411
Epoch: 10, Loss: 0.6240401268005371
Epoch: 11, Loss: 0.6128078699111938
Epoch: 12, Loss: 0.6014628410339355
Epoch: 13, Loss: 0.5900222659111023
Epoch: 14, Loss: 0.5784705877304077
Epoch: 15, Loss: 0.5668260455131531
Epoch: 16, Loss: 0.5551018118858337
Epoch: 17, Loss: 0.5432732105255127
Epoch: 18, Loss: 0.5313003659248352
Epoch: 19, Loss: 0.519077718257904
Epoch: 20, Loss: 0.506550669670105
Epoch: 21, Loss: 0.4936997592449188
Epoch: 22, Loss: 0.4804483950138092
Epoch: 23, Loss: 0.46678170561790466
Epoch: 24, Loss: 0.4526923894882202
Epoch: 25, Loss: 0.4382525086402893


In [104]:
# Print the model weights and biases
print('Model weights:')
print(model.network[0].weight)
print('Model biases:')
print(model.network[0].bias)

Model weights:
Parameter containing:
tensor([[ 6.3411e-02,  1.9359e-01,  2.7810e-01,  2.9075e-01,  1.6261e-01,
          1.8396e-01,  1.7374e-01,  3.5223e-01,  1.9172e-01, -1.6790e-01,
          5.0785e-02,  6.2013e-02,  1.2589e-01,  2.2212e-01, -1.1851e-01,
          2.5298e-04, -7.5342e-02,  1.2878e-01, -1.8251e-02, -3.0675e-02,
          2.4538e-01,  3.1325e-01,  3.9837e-01,  2.2699e-01,  1.9796e-01,
          3.4310e-01,  1.8602e-01,  3.7394e-01,  4.0327e-01,  1.2336e-01],
        [ 2.1009e-01,  1.4024e-01,  3.1556e-01,  4.2080e-01,  2.3716e-01,
          6.7354e-02,  1.3016e-01,  1.6799e-01,  1.6839e-01, -1.8712e-01,
          8.9035e-02,  1.7247e-02,  2.3819e-01,  2.1984e-01, -2.3849e-01,
         -1.0597e-01, -3.9944e-05,  1.7837e-01,  2.3280e-02, -1.8507e-01,
          2.3217e-01,  7.4617e-02,  1.7397e-01,  3.1727e-01,  3.4675e-01,
          3.8490e-01,  2.7170e-01,  2.2621e-01,  1.5601e-01,  2.5634e-01],
        [-2.0629e-01, -2.5296e-01,  2.6603e-02, -1.0260e-01, -2.7529e-01,

### **Model Evaluation**

In [110]:
# Make predictions on testing data
with torch.no_grad():
    y_pred = model(X_test_tensor)

# Calculate the accuracy using torchmetrics
accuracy = Accuracy(task='binary')
print('Accuracy on testing data:', accuracy(y_pred.squeeze(), y_test_tensor).item())

Accuracy on testing data: 0.9649122953414917
